# 基于langchain创建自己专属的对话大模型

1. 领域精准问答
2. 数据更新频繁
3. 生成内容可解释可追溯
4. 数据隐私保护

通过这个例子，我们将基于`LangChain`, `OpenAI(LLM)`,  `vector DB`构建一个属于自己的LLM模型。

主要使用的技术————***Retrieval Augmented Generation (RAG)***

首先确保自己拥有一个 `OpenAI API key` (也并非必须)


### 准备环境

In [1]:
! pip install -qU \
    langchain==0.0.316 \
    openai==0.28.1  \
    tiktoken==0.5.1  \
    cohere \
    chromadb==0.4.15

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Checking for Rust toolchain....
      
      Cargo, the Rust package manager, is not installed or is not on PATH.
      This package requires Rust and Cargo to compile extensions. Install it through
      the system's package manager or via https://rustup.rs/
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


### 创建一个对话模型(no RAG)

In [8]:
import os
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

# 加载 .env 文件
load_dotenv()

# 从环境变量中获取 API_KEY 和 BASE_URL
API_KEY = os.getenv("API_KEY")
BASE_URL = os.getenv("BASE_URL")

# 确认 API_KEY 和 BASE_URL 已正确加载
if not API_KEY or not BASE_URL:
    raise ValueError("API_KEY 或 BASE_URL 未正确加载")

# 创建 ChatOpenAI 实例
chat = ChatOpenAI(
    openai_api_key=API_KEY,
    model_kwargs={"base_url": BASE_URL}
)

OpenAI Python 的例子
```python
[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Knock knock."},
    {"role": "assistant", "content": "Who's there?"},
    {"role": "user", "content": "Orange."},
]
```
https://cookbook.openai.com/examples/how_to_format_inputs_to_chatgpt_models


但是langchain 需要使用以下的格式

In [11]:

from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)


messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Knock knock."),
    AIMessage(content="Who's there?"),
    HumanMessage(content="Orange"),
    
]
print(messages)

[SystemMessage(content='You are a helpful assistant.'), HumanMessage(content='Knock knock.'), AIMessage(content="Who's there?"), HumanMessage(content='Orange')]


In [10]:
res = chat(messages)
res

AttributeError: module 'openai' has no attribute 'error'

因为 `res`也是`AIMessage`属性，所以我们可以直接进行添加，即可实现下一次的响应

In [4]:
messages.append(res)
res = chat(messages)

print(res.content)

Orange you glad I'm here to assist you?


#### 处理LLM存在的缺陷
1. 容易出现幻觉
2. 信息滞后
3. 专业领域深度知识匮乏


In [5]:
messages = [
    SystemMessage(content="你是一个专业的知识助手。"),
    HumanMessage(content="你知道baichuan2模型吗？"),
]

In [6]:
res = chat(messages)
print(res.content)


是的，我了解baichuan2模型。baichuan2是一种基于深度学习的图像分割模型，主要用于图像语义分割任务。它采用了编码-解码的架构，通过卷积神经网络提取图像特征，并将特征映射到与输入图像大小相同的分割结果。baichuan2模型在许多图像分割比赛和任务中取得了很好的性能，并被广泛应用于医学图像分割、自动驾驶、图像编辑等领域。


chatgpt AI无法满足我们在某些特定领域的专业需求，我们可以通过知识注入的方式，利用prompt来解决这种问题：

In [ ]:

baichuan2_information = [
    "Baichuan 2是一个大规模多语言语言模型，它专注于训练在多种语言中表现优异的模型，包括不仅限于英文。这使得Baichuan 2在处理各种语言的任务时能够取得显著的性能提升。",
    "Baichuan 2是从头开始训练的，使用了包括了2.6万亿个标记的庞大训练数据集。相对于以往的模型，Baichuan 2提供了更丰富的数据资源，从而能够更好地支持多语言的开发和应用。",
    "Baichuan 2不仅在通用任务上表现出色，还在特定领域（如医学和法律）的任务中展现了卓越的性能。这为特定领域的应用提供了强有力的支持。"
]

source_knowledge = "\n".join(baichuan2_information)

In [ ]:
print(source_knowledge)


In [ ]:
query = "你知道baichuan2模型吗？"

prompt_template = f"""基于以下内容回答问题：

内容:
{source_knowledge}

Query: {query}"""

In [ ]:
prompt = HumanMessage(
    content=prompt_template
)
messages.append(prompt)

res = chat(messages)

In [ ]:
print(res.content)


当我们注入一些专业的知识后，模型就能够很好的回答相关问题。
如果每一个问题都去用相关的外部知识进行增强拼接的话，那么回答的准确性就大大增加？？？？

### 创建一个RAG对话模型

#### 1. 加载数据 （以baichuan2论文为例）

   https://arxiv.org/pdf/2309.10305v2.pdf

In [ ]:
! pip install pypdf

In [12]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("https://arxiv.org/pdf/2309.10305.pdf")

pages = loader.load_and_split()

ImportError: cannot import name 'create_importer' from 'langchain._api' (d:\Anaconda\envs\langchain\lib\site-packages\langchain\_api\__init__.py)

In [ ]:
pages[0]

#### 2. 知识切片 将文档分割成均匀的块。每个块是一段原始文本

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
)

docs = text_splitter.split_documents(pages)

In [ ]:
len(docs)



#### 3. 利用embedding模型对每个文本片段进行向量化，并储存到向量数据库中

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma


embed_model = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents=docs, embedding=embed_model , collection_name="openai_embed")

#### 4. 通过向量相似度检索和问题最相关的K个文档。

In [ ]:
query = "How large is the baichuan2 vocabulary?"
result = vectorstore.similarity_search(query ,k = 2)

In [ ]:
result

#### 5. 原始`query`与检索得到的文本组合起来输入到语言模型，得到最终的回答

In [ ]:
def augment_prompt(query: str):
  # 获取top3的文本片段
  results = vectorstore.similarity_search(query, k=3)
  source_knowledge = "\n".join([x.page_content for x in results])
  # 构建prompt
  augmented_prompt = f"""Using the contexts below, answer the query.

  contexts:
  {source_knowledge}

  query: {query}"""
  return augmented_prompt

In [ ]:
print(augment_prompt(query))

In [ ]:
# 创建prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)

messages.append(prompt)

res = chat(messages)

print(res.content)

### 没有OPENAI api key怎么办 创建一个非openai的对话模型  


1.   embedding模型  
2.   chat模型


In [ ]:
! pip install sentence-transformers

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

model_name = "sentence-transformers/sentence-t5-large"

In [ ]:
embedding = HuggingFaceEmbeddings(model_name=model_name)

In [ ]:
vectorstore_hf = Chroma.from_documents(documents=docs, embedding=embedding , collection_name="huggingface_embed")

In [ ]:
result = vectorstore_hf.similarity_search(query ,k = 2)


In [ ]:
print(result)


通过本地部署的模型进行交互